# MERFISH Data reorganization

In [1]:
import pandas as pd
import numpy as np
import os, sys, time
import re
import shutil
from importlib import reload
# this version doesn't have ChromAn compiled, so directly load from src:
sys.path.append(r"/lab/weissman_imaging/puzheng/Softwares/ChromAn")
import src

In [2]:
from src.file_io.data_organization import search_fovs_in_folders, Color_Usage
data_home =  r"/lab/weissman_imaging/puzheng"

data_folder = os.path.join(data_home, 'msBrain', '20230924-msBrain0913_MF1+MO40919')

fds, fovs = search_fovs_in_folders(data_folder)


- searching in folder: /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919
-- 17 folders, 725 fovs detected.


In [4]:
# Color usage file marks the organization of imaging files
color_usage_filename = os.path.join(data_folder, 'Analysis', 'color_usage_MF1.csv')
color_usage_df = Color_Usage(color_usage_filename)

- load color_usage from file: /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/Analysis/color_usage_MF1.csv


In [5]:
date = os.path.basename(data_folder).split('-')[0]

sample = 'msBrain'
library = 'MF1'

print(date, sample, library)

20230924 msBrain MF1


In [6]:
# load data_organization and match:
sys.path.append(r"/lab/weissman_imaging/puzheng/Softwares")
from ChromAn.src.file_io.data_organization import Data_Organization, search_fovs_in_folders

data_organization_filename = r'../../merlin_parameters/dataorganization/20230924-MF1_msBrain_22bit.csv'
_do = Data_Organization(data_organization_filename,)

- load color_usage from file: ../../merlin_parameters/dataorganization/20230924-MF1_msBrain_22bit.csv


In [7]:
from src.file_io.data_organization import create_folder
# shutter files
ref_image_type = _do.loc[0,'imageType'] #'748_637_477_405_s13_n1000' # ref 58 / 65
polyt_image_type = _do.loc[_do['channelName']=='PolyT','imageType'].values[0] #'748_637_477_s13_n1000'
image_type = _do.loc[4,'imageType'] #'748_637_477_s13_n1000' # ref 36/39

# fiducial beads
#ref_channel = 488
#ref_frame = 13 # start from 0!

# target folder
merfish_data_folder = r'/lab/weissman_imaging/puzheng/MERFISH_data'

# target folder:
target_folder = os.path.join(merfish_data_folder, f'{date}-{sample}_{library}')
create_folder(target_folder)


Creating folder: /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1


In [9]:
#sel_fovs = []
folder_regexp = r'H([0-9]+)[MRCP]([0-9+])'

ref_round = 0
polyt_round = _do.loc[_do['channelName']=='PolyT','imagingRound'].values[0]
#fov_ids = np.arange(2)
overwrite = True
remove_source = False

sel_fovs = np.arange(0,101)

for _fov_id, _fov_name in enumerate(fovs):
    # check if this fov is selected:
    if 'sel_fovs' in locals() and _fov_id not in sel_fovs:
        continue # skip if not selected
    # prceed for the rest:
    for _fd in fds:
        # match round and hyb
        _match = re.search(folder_regexp, os.path.basename(_fd))
        if _match is None:
            continue
        # if not exist in color_usage, skip
        if os.path.basename(_fd) not in list(color_usage_df.index):
            continue
        _round, _region = _match.groups()
        # mutliple files for each image:
        _files = [os.path.join(_fd, _fl) for _fl in os.listdir(_fd) if _fov_name.split(os.extsep)[0] in _fl]
        # reference round specified
        if int(_round) == ref_round:
            _image_type = ref_image_type
        elif int(_round) == polyt_round:
            _image_type = polyt_image_type
        else:
            _image_type = image_type
    
        # target file
        _target_files = [os.path.join(target_folder, f"{_image_type}_{_fov_id}_{_round}{os.extsep}{_fl.split(os.extsep)[-1]}") 
                         for _fl in _files]
        for _fl, _tar_fl in zip(_files, _target_files):
            if remove_source:
                print(f"move {_fl} to {_tar_fl}")
                shutil.move(_fl, _tar_fl)
            else:
                if not os.path.isfile(_tar_fl) or overwrite:
                    print(f"copy {_fl} to {_tar_fl}")
                    shutil.copyfile(_fl, _tar_fl)
                else:
                    print(f"{_tar_fl} already exist, skip")

copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H0M1/Conv_zscan_000.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_405_s13_0_0.xml
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H0M1/Conv_zscan_000.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_405_s13_0_0.dax
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H0M1/Conv_zscan_000.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_405_s13_0_0.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H0M1/Conv_zscan_000.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_405_s13_0_0.off
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H0M1/Conv_zscan_000.power to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_405_s13_0_0.power
c

copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H16M17/Conv_zscan_000.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_0_16.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H16M17/Conv_zscan_000.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_0_16.off
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H16M17/Conv_zscan_000.power to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_0_16.power
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H0M1/Conv_zscan_001.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_405_s13_1_0.dax
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H0M1/Conv_zscan_001.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_405_s13_1_0.inf
copy

copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H16M17/Conv_zscan_001.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_1_16.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H16M17/Conv_zscan_001.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_1_16.off
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H16M17/Conv_zscan_001.power to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_1_16.power
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H16M17/Conv_zscan_001.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_1_16.xml
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H0M1/Conv_zscan_002.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_405_s13_2_0.xml
copy 

copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H16M17/Conv_zscan_002.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_2_16.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H16M17/Conv_zscan_002.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_2_16.off
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H16M17/Conv_zscan_002.power to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_2_16.power
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H0M1/Conv_zscan_003.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_405_s13_3_0.xml
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H0M1/Conv_zscan_003.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_405_s13_3_0.dax
copy

copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H16M17/Conv_zscan_003.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_3_16.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H16M17/Conv_zscan_003.power to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_3_16.power
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H16M17/Conv_zscan_003.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_3_16.off
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H0M1/Conv_zscan_004.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_405_s13_4_0.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H0M1/Conv_zscan_004.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_405_s13_4_0.dax
copy

...

copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H1M2/Conv_zscan_096.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_96_1.xml
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H1M2/Conv_zscan_096.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_96_1.off
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H1M2/Conv_zscan_096.power to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_96_1.power
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H1M2/Conv_zscan_096.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_96_1.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H2M3/Conv_zscan_096.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_96_2.dax
copy /lab/weissm

copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H1M2/Conv_zscan_097.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_97_1.xml
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H1M2/Conv_zscan_097.power to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_97_1.power
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H1M2/Conv_zscan_097.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_97_1.off
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H1M2/Conv_zscan_097.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_97_1.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H2M3/Conv_zscan_097.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_97_2.dax
copy /lab/weissm

copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H1M2/Conv_zscan_098.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_98_1.off
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H1M2/Conv_zscan_098.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_98_1.xml
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H1M2/Conv_zscan_098.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_98_1.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H2M3/Conv_zscan_098.power to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_98_2.power
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H2M3/Conv_zscan_098.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_98_2.dax
copy /lab/weissm

copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H1M2/Conv_zscan_099.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_99_1.xml
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H1M2/Conv_zscan_099.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_99_1.off
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H2M3/Conv_zscan_099.power to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_99_2.power
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H2M3/Conv_zscan_099.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_99_2.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H2M3/Conv_zscan_099.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_99_2.dax
copy /lab/weissm

copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H1M2/Conv_zscan_100.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_100_1.off
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H1M2/Conv_zscan_100.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_100_1.xml
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H1M2/Conv_zscan_100.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_100_1.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H1M2/Conv_zscan_100.power to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_100_1.power
copy /lab/weissman_imaging/puzheng/msBrain/20230924-msBrain0913_MF1+MO40919/H2M3/Conv_zscan_100.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20230924-msBrain_MF1/748_637_545_477_s13_100_2.dax
copy /lab/w

## Check file length

In [21]:
# double check if filenumber equals to expected:
print(len(os.listdir(target_folder)), len(color_usage_df) * len(fovs) * 5)

25600 23880


# Generate data_organization

go to jupyter: https://c4b15.wi.mit.edu/user/puzheng/notebooks/lab/weissman_imaging/puzheng/Softwares/Weissman_MERFISH_Scripts/MERFISH_preprocess/Data_organization/20230830-Generate_dataOrganization.ipynb

In [ ]:
## Automatic generate data_organization
from src.file_io.data_organization import Color_Usage

In [ ]:
def generate_dataOrganization():
    

